In [7]:
def preprocess_text(text):
    """Preprocess text by keeping all characters as literals"""
    # Convert escape sequences to literal characters
    processed = text.encode('unicode_escape').decode('utf-8')
    return processed

In [8]:
def caesar_cipher(text, alphabet, shift, mode='encrypt'):
    """Encrypts or decrypts text using the Caesar cipher."""
    result = ""
    for char in text:
        if char in alphabet:
            index = alphabet.index(char)
            if mode == 'encrypt':
                new_index = (index + shift) % len(alphabet)
            else:  # mode == 'decrypt'
                new_index = (index - shift) % len(alphabet)
            result += alphabet[new_index]
        else:
            result += char
    return result

In [9]:
def encode_key(key, alphabet):
    """Encodes the key to a string using the given alphabet (base conversion)."""
    base = len(alphabet)
    if key == 0:
        return alphabet[0]
    
    digits = []
    while key:
        digits.append(alphabet[key % base])
        key //= base
    return "".join(digits[::-1])  # Reverse to get correct order

def decode_key(encoded_key, alphabet):
    """Decodes the key from a string to an integer."""
    base = len(alphabet)
    key = 0
    for char in encoded_key:
        key = key * base + alphabet.index(char)
    return key

def encrypt_with_key(text, alphabet, shift, handle_escapes=False):
    """Encrypts text and encodes the key within the ciphertext."""
    if handle_escapes:
        text = text.encode('unicode_escape').decode('utf-8')
    encrypted_text = caesar_cipher(text, alphabet, shift, mode='encrypt')
    encoded_key = encode_key(shift, alphabet)
    return f"{encoded_key}|{encrypted_text}"

def decrypt_with_key(encrypted_message, alphabet):
    """Decrypts a message containing an encoded key."""
    try:
        encoded_key, encrypted_text = encrypted_message.split("|", 1)
        shift = decode_key(encoded_key, alphabet)
        return caesar_cipher(encrypted_text, alphabet, shift, mode='decrypt')
    except ValueError:
        return "Invalid encrypted message format."


# Example usage:
shift_key = 3
plaintext = "pneumonoultramicroscopicsilicovolcanoconiosis"
processed_plaintext = preprocess_text(plaintext)
alphabet = r"`,.pyfgcrl/=\aoeuidhtns-;qjkxbmwvz"
processed_alphabet = preprocess_text(alphabet)

# Encrypt
encrypted = encrypt_with_key(processed_plaintext, processed_alphabet, shift_key)
print(f"Encrypted: {encrypted}")

# Decrypt
decrypted = decrypt_with_key(encrypted, processed_alphabet)
print(f"Decrypted: {decrypted}")

Encrypted: p|g;dhzi;ih\-=uzt/=iq/igt/qt\t/i,i\/u;i/i;tiqtq
Decrypted: pneumonoultramicroscopicsilicovolcanoconiosis
